In [4]:
import pickle

# Cargar el modelo desde el pickle
with open("lgb_model_2023.0.pkl", "rb") as f:
    modelo = pickle.load(f)

In [5]:
import pandas as pd
import plotly.express as px

# Importancia de características
importancia = modelo.feature_importances_
columnas = ["Variable1", "Variable2", "Variable3"]  # Reemplaza con tus nombres reales

# Crear un DataFrame
importancia_df = pd.DataFrame({
    "Característica": columnas,
    "Importancia": importancia
}).sort_values(by="Importancia", ascending=False)

fig_importancia = px.bar(importancia_df, x="Importancia", y="Característica", orientation="h",
                         title="Importancia de Características", labels={"Importancia": "Valor"})


AttributeError: 'Pipeline' object has no attribute 'feature_importances_'

In [ ]:
import pickle
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.pipeline import Pipeline
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, classification_report
import numpy as np
import pandas as pd

years = sorted(data_filter_imputed['Fecha de Corte'].unique())
label_encoder = LabelEncoder()
# Crear un diccionario para almacenar los resultados
results = {
    'Year': [],
    'Accuracy': [],
    'ROC AUC': [],
    'F1 Score': [],
    'Precision': [],
    'Recall': [],
    'Confusion Matrix': [],
    'Classification Report': []
}

# Ciclo para la validación secuencial por año
for i in range(1, len(years)):  # Comenzamos desde el segundo año
    # Usamos los años anteriores para entrenamiento y el año actual para prueba
    train_years = years[:i]  # Datos de entrenamiento hasta el año i-1
    test_year = years[i]     # Año para prueba

    # Filtrar los datos de entrenamiento y prueba
    X_train = data_filter_imputed[data_filter_imputed['Fecha de Corte'].isin(train_years)].drop(columns=exclude_columns)
    y_train = data_filter_imputed[data_filter_imputed['Fecha de Corte'].isin(train_years)]['FDC_FLC_categoria']
    X_test = data_filter_imputed[data_filter_imputed['Fecha de Corte'] == test_year].drop(columns=exclude_columns)
    y_test = data_filter_imputed[data_filter_imputed['Fecha de Corte'] == test_year]['FDC_FLC_categoria']

    # Codificar las etiquetas de la variable objetivo
    y_train_encoded = label_encoder.fit_transform(y_train)
    y_test_encoded = label_encoder.transform(y_test)

    # Pipeline de XGBoost como modelo base
    xgb_model = xgb.XGBClassifier(objective='multi:softprob', num_class=len(np.unique(y_train_encoded)), eval_metric='mlogloss')

    # Crear el pipeline con preprocesador y el clasificador
    pipeline_xgb = Pipeline(steps=[
        ('preprocessor', preprocessor),  # Preprocesamiento
        ('classifier', xgb_model)  # Clasificador XGBoost
    ])

    # Entrenamiento de XGBoost
    pipeline_xgb.fit(X_train, y_train_encoded)

    # Predicciones de XGBoost
    y_pred_xgb_proba = pipeline_xgb.predict_proba(X_test)  # Probabilidades de XGBoost
    
    # Calcular la curva ROC y AUC
    roc_auc = roc_auc_score(y_test_encoded, y_pred_xgb_proba, multi_class='ovr')

    # Calcular los fpr y tpr para la clase positiva (como ejemplo)
    fpr, tpr, _ = roc_curve(y_test_encoded, y_pred_xgb_proba[:, 1], pos_label=1)

    # Graficar la curva ROC
    plt.figure(figsize=(6, 6))
    plt.plot(fpr, tpr, color='b', label=f'AUC - {test_year} = {roc_auc:.2f}')
    plt.plot([0, 1], [0, 1], color='k', linestyle='--')
    plt.title(f'Curva ROC AUC para {test_year}')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend(loc='lower right')

    # Guardar la figura en un archivo pickle
    pickle_filename = f'auc_curve_{test_year}.pkl'
    with open(pickle_filename, 'wb') as f:
        pickle.dump(plt, f)

    # Cerrar la figura para liberar memoria
    plt.close()

    # Guardar las métricas en los resultados
    results['Year'].append(test_year)
    results['ROC AUC'].append(roc_auc)

# Imprimir los resultados finales
print("\nResultados finales:")
print(results)


In [ ]:
import matplotlib.pyplot as plt
import pickle
from sklearn.metrics import roc_curve, roc_auc_score

# Lista de años disponibles y diccionario para las predicciones por año
years = [2017, 2018, 2019, 2020, 2021, 2022, 2023]
y_true_dict = {}  # Diccionario de etiquetas verdaderas (reales) por año
y_pred_proba_dict = {}  # Diccionario de probabilidades predichas por modelo por año

# Asegúrate de cargar los datos para cada año (ya precomputados)
# y_true_dict y y_pred_proba_dict deben estar disponibles
# Ejemplo de estructura:
# y_true_dict[2017] = array([0, 1, 1, 0, 1, ...])  # Etiquetas verdaderas para 2017
# y_pred_proba_dict[2017] = array([[0.2, 0.8], [0.7, 0.3], ...])  # Probabilidades predichas para 2017

for year in years:
    # Obtener etiquetas verdaderas y probabilidades predichas para el año actual
    y_true = y_true_dict[year]
    y_pred_proba = y_pred_proba_dict[year]

    # Calcular la curva ROC
    fpr, tpr, _ = roc_curve(y_true, y_pred_proba[:, 1], pos_label=1)
    roc_auc = roc_auc_score(y_true, y_pred_proba[:, 1])

    # Crear la gráfica de la curva ROC
    plt.figure(figsize=(6, 6))
    plt.plot(fpr, tpr, color='b', label=f'AUC = {roc_auc:.2f}')
    plt.plot([0, 1], [0, 1], color='k', linestyle='--', label='Random')
    plt.title(f'Curva ROC para {year}')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend(loc='lower right')

    # Guardar la gráfica como archivo pickle
    pickle_filename = f"auc_curve_{year}.pkl"
    with open(pickle_filename, "wb") as f:
        pickle.dump(plt.gcf(), f)

    # Cerrar la figura para evitar sobrecarga
    plt.close()

    print(f"Gráfica ROC AUC para {year} guardada en {pickle_filename}")
